<a href="https://colab.research.google.com/github/Hannah1011/seoul_data/blob/main/traffic_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Drive랑 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# 필요한 라이브러리 import
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## 2. CrossWalk 전처리

In [3]:
# 엑셀 파일을 불러오기
file_path = '/content/drive/MyDrive/seoul_data/서울시 교통안전시설물 횡단보도 정보 (1).xlsx'

# 첫 번째 시트와 두 번째 시트를 불러오기
df1 = pd.read_excel(file_path, sheet_name=0)  # 첫 번째 시트
df2 = pd.read_excel(file_path, sheet_name='Sheet1')  # 두 번째 시트

# 두 번째 시트의 구코드와 동코드를 기준으로 동이름을 첫 번째 시트에 추가
df1['동이름'] = df1.apply(
    lambda row: df2[(df2['구코드'] == row['구코드']) & (df2['동코드'] == row['동코드'])]['동이름'].values[0]
    if not df2[(df2['구코드'] == row['구코드']) & (df2['동코드'] == row['동코드'])].empty else None, axis=1
)

# 구이름이 '종로구'이고 동이름이 '세종로'인 행의 구코드와 동코드 찾기
filtered_row = df1[(df1['구이름'] == '종로구') & (df1['동이름'] == '세종로')]
if not filtered_row.empty:
    gu_code = filtered_row['구코드'].values[0]
    dong_code = filtered_row['동코드'].values[0]
    print(f"종로구 세종로의 구코드: {gu_code}, 동코드: {dong_code}")
else:
    print("종로구 세종로의 행을 찾을 수 없습니다.")


종로구 세종로의 구코드: 110.0, 동코드: 11900.0


In [4]:
# 첫 번째 시트만 csv 파일로 저장
df1.to_csv('/content/drive/MyDrive/seoul_data/crosswalk.csv', index=False)

# 각 열마다 na 값이 몇 개인지 계산
na_counts = df1.isna().sum()

# na 값 개수 출력
print(na_counts)

상태              8
횡단보도종류코드        9
구코드           727
구이름           727
동코드           302
동이름          1854
도로구분         1494
공사형태        12670
dtype: int64


In [6]:
# 1. '상태'가 NA인 행 제거
df_cleaned = df1.dropna(subset=['상태'])

# 2. '구코드'가 NA인 행 출력
gu_na_rows = df_cleaned[df_cleaned['구코드'].isna()]
print("구코드가 NA인 행들:")
print(gu_na_rows)

구코드가 NA인 행들:
         상태  횡단보도종류코드  구코드  구이름  동코드   동이름  도로구분 공사형태
516     1.0       2.0  NaN  NaN  NaN  None   2.0  004
547     1.0       2.0  NaN  NaN  0.0  None   2.0  NaN
548     1.0       2.0  NaN  NaN  NaN  None   2.0  004
549     1.0       2.0  NaN  NaN  0.0  None   2.0  NaN
550     1.0       2.0  NaN  NaN  NaN  None   2.0  004
...     ...       ...  ...  ...  ...   ...   ...  ...
121242  1.0       1.0  NaN  NaN  NaN  None   2.0  004
121494  1.0       1.0  NaN  NaN  NaN  None   2.0  004
121563  1.0       2.0  NaN  NaN  0.0  None   2.0  NaN
121575  1.0       4.0  NaN  NaN  NaN  None   1.0  001
121589  1.0       3.0  NaN  NaN  NaN  None   1.0  004

[727 rows x 8 columns]


In [7]:
df_cleaned = df1.dropna(subset=['구코드'])
# 3. '도로구분'에서 나올 수 있는 값들 출력
unique_road_types = df_cleaned['도로구분'].unique()
print("\n도로구분에 나올 수 있는 값들:")
print(unique_road_types)


도로구분에 나올 수 있는 값들:
[ 2.  1. nan]


In [8]:
# 도로구분 필요없음
# 2. '도로구분' 열 삭제
df_cleaned = df_cleaned.drop(columns=['도로구분'])

In [11]:
# 각 열마다 na 값이 몇 개인지 계산
na_counts = df_cleaned.isna().sum()
print(na_counts)

상태              8
횡단보도종류코드        9
구코드             0
구이름             0
동코드             1
동이름          1127
공사형태        12352
dtype: int64


In [12]:
unique_repair_types = df_cleaned['공사형태'].unique()
print("\n공사형태에 나올 수 있는 값들:")
print(unique_repair_types)


공사형태에 나올 수 있는 값들:
['005' nan '004' '003' '001' '010' ' ' '002' '006' '009' '008']


In [13]:
# 2. '동이름'이 NA인 행 출력
dong_na_rows = df_cleaned[df_cleaned['동이름'].isna()]
print("동이름이 NA인 행들:")
print(dong_na_rows)

동이름이 NA인 행들:
         상태  횡단보도종류코드    구코드  구이름      동코드   동이름 공사형태
5623    1.0       3.0  380.0  은평구  10800.0  None  004
5624    1.0       3.0  380.0  은평구  10800.0  None  004
5625    1.0       3.0  380.0  은평구  10800.0  None  004
5626    1.0       3.0  380.0  은평구  10800.0  None  004
7540    1.0       1.0  380.0  은평구  10800.0  None  005
...     ...       ...    ...  ...      ...   ...  ...
121539  1.0       2.0  440.0  마포구      0.0  None  003
121542  1.0       2.0  110.0  종로구      0.0  None  001
121544  1.0       1.0  540.0  금천구      0.0  None  001
121552  1.0       2.0  590.0  동작구      0.0  None  NaN
121555  1.0       1.0  530.0  구로구      0.0  None  NaN

[1127 rows x 7 columns]


In [14]:
# 3. '구이름'과 '동코드'의 유니크한 조합 추출
unique_combinations = dong_na_rows[['구이름', '동코드']].drop_duplicates()

print("동이름이 NA인 행들 중 유니크한 구이름과 동코드 조합:")
print(unique_combinations)


동이름이 NA인 행들 중 유니크한 구이름과 동코드 조합:
         구이름      동코드
5623     은평구  10800.0
8590     은평구      0.0
11842   서대문구      NaN
11907   서대문구  18000.0
12588   서대문구  18700.0
12764   서대문구  17200.0
13463     중구  17700.0
13616   서대문구  16900.0
14008     중구      0.0
19335    금천구      0.0
20544    구로구      0.0
23075    구로구  13300.0
25460    서초구      0.0
34590    강동구  11100.0
34907    강동구      0.0
38706    광진구      0.0
39092    광진구  11500.0
40855    성동구  16500.0
41007    성동구  16200.0
41884   동대문구  12200.0
43893    강서구      0.0
45727    종로구      0.0
48016    성북구  10200.0
48065    성북구  17400.0
48107    강북구  13400.0
49086   동대문구  12500.0
51824    서초구  11800.0
51905    성북구      0.0
53851    중랑구      0.0
58481    강남구      0.0
59107    성동구      0.0
60220    성북구  17500.0
62437    도봉구  10300.0
64470    마포구      0.0
68569    용산구      0.0
68738    송파구      0.0
68813    강북구      0.0
68914   영등포구      0.0
69115    양천구      0.0
69617    노원구      0.0
69628    도봉구      0.0
69796    관악구      0.0
70341    동작구      0.0


In [15]:
# 1. '은평구'이고 동코드가 10800인 행의 동이름을 '역촌동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '은평구') & (df_cleaned['동코드'] == 10800), '동이름'] = '역촌동'

# 2. 동코드가 0.0인 행들의 구이름을 '서대문구', 동이름을 '창천동'으로 변경
df_cleaned.loc[df_cleaned['동코드'] == 0.0, ['구이름', '동이름']] = ['서대문구', '창천동']

# 3. 동코드가 NaN인 행 삭제
df_cleaned = df_cleaned.dropna(subset=['동코드'])

# 4. 서대문구 동코드가 18000이면 구이름을 '종로구', 동이름을 '교북동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '서대문구') & (df_cleaned['동코드'] == 18000), ['구이름', '동이름']] = ['종로구', '교북동']

# 5. 서대문구 동코드가 18700이면 구이름을 '종로구', 동이름을 '무악동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '서대문구') & (df_cleaned['동코드'] == 18700), ['구이름', '동이름']] = ['종로구', '무악동']

# 6. 서대문구 동코드가 17200이면 구이름을 '종로구', 동이름을 '명륜4가'로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '서대문구') & (df_cleaned['동코드'] == 17200), ['구이름', '동이름']] = ['종로구', '명륜4가']

# 7. 중구 동코드가 17700이면 구이름을 '종로구', 동이름을 '평동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '중구') & (df_cleaned['동코드'] == 17700), ['구이름', '동이름']] = ['종로구', '평동']

# 8. 서대문구 동코드가 16900인 행 삭제
df_cleaned = df_cleaned[~((df_cleaned['구이름'] == '서대문구') & (df_cleaned['동코드'] == 16900))]

# 9. 성북구 동코드가 17500이면 구이름을 '종로구', 동이름을 '숭인동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '성북구') & (df_cleaned['동코드'] == 17500), ['구이름', '동이름']] = ['종로구', '숭인동']

# 10. 동대문구 동코드가 17500이면 구이름을 '종로구', 동이름을 '숭인동'으로 변경
df_cleaned.loc[(df_cleaned['구이름'] == '동대문구') & (df_cleaned['동코드'] == 17500), ['구이름', '동이름']] = ['종로구', '숭인동']

# 11. '동이름'이 아직도 NA인 행 출력
dong_na_rows_after = df_cleaned[df_cleaned['동이름'].isna()]
print("동이름이 아직 NA인 행들:")
print(dong_na_rows_after)

동이름이 아직 NA인 행들:
         상태  횡단보도종류코드    구코드   구이름      동코드   동이름 공사형태
23075   1.0       2.0  530.0   구로구  13300.0  None  001
24860   1.0       2.0  530.0   구로구  13300.0  None  005
34590   1.0       2.0  740.0   강동구  11100.0  None  005
39092   1.0       2.0  210.0   광진구  11500.0  None  005
39093   1.0       2.0  210.0   광진구  11500.0  None  005
39094   1.0       2.0  210.0   광진구  11500.0  None  005
39453   1.0       2.0  740.0   강동구  11100.0  None  004
39454   1.0       3.0  740.0   강동구  11100.0  None  004
40855   1.0       2.0  200.0   성동구  16500.0  None  005
40856   1.0       2.0  200.0   성동구  16500.0  None  005
40857   1.0       2.0  200.0   성동구  16500.0  None  005
40921   1.0       2.0  200.0   성동구  16500.0  None  005
40922   1.0       2.0  200.0   성동구  16500.0  None  005
40923   1.0       2.0  200.0   성동구  16500.0  None  005
40924   1.0       2.0  200.0   성동구  16500.0  None  005
40925   1.0       2.0  200.0   성동구  16500.0  None  005
40926   1.0       2.0  200.0   성동구  16500.0  None

In [17]:
dong_na_count = df_cleaned['동이름'].isna().sum()
print(f"동이름이 NA인 행의 개수: {dong_na_count}")

동이름이 NA인 행의 개수: 46


In [18]:
na_counts = df_cleaned.isna().sum()
print("각 열의 결측치 개수:")
print(na_counts)

각 열의 결측치 개수:
상태              8
횡단보도종류코드        9
구코드             0
구이름             0
동코드             0
동이름            46
공사형태        12352
dtype: int64


In [24]:
# 공사형태가 공백(' ')인 행을 삭제
df_cleaned = df_cleaned[df_cleaned['공사형태'] != ' ']

In [32]:
# 횡단보도종류코드가 5 또는 6인 행을 제거
df_cleaned = df_cleaned[~df_cleaned['횡단보도종류코드'].isin([5, 6])]

In [20]:
df_cleaned = df_cleaned.dropna(subset=['상태', '횡단보도종류코드', '동이름', '공사형태'])
# '구이름'과 '동이름'을 합쳐서 새로운 열 '구_동' 생성
df_cleaned['구_동'] = df_cleaned['구이름'] + ' ' + df_cleaned['동이름']

결측치 제거 후 총 행의 개수: 108464


In [33]:
total_rows = df_cleaned.shape[0]
print(f"결측치 제거 후 총 행의 개수: {total_rows}")

결측치 제거 후 총 행의 개수: 108187


In [34]:
# 'cleaned_crosswalk.csv'로 저장
df_cleaned.to_csv('/content/drive/MyDrive/seoul_data/cleaned_crosswalk.csv', index=False)

In [36]:
# 1. 상태별 열을 생성하여 '구_동'별로 상태 개수 계산
df_status = df_cleaned.pivot_table(
    index='구_동',
    columns='상태',
    aggfunc='size',
    fill_value=0
).reset_index()

# 2. 횡단보도종류코드별 열을 생성하여 '구_동'별로 횡단보도종류코드 개수 계산
df_crosswalk_type = df_cleaned.pivot_table(
    index='구_동',
    columns='횡단보도종류코드',
    aggfunc='size',
    fill_value=0
).reset_index()

# 3. 공사형태별 열을 생성하여 '구_동'별로 공사형태 개수 계산
df_construction_type = df_cleaned.pivot_table(
    index='구_동',
    columns='공사형태',
    aggfunc='size',
    fill_value=0
).reset_index()

# 4. 다른 열들(횡단보도 개수, 횡단보도종류코드 개수, 공사형태 개수)도 구_동별로 그룹화하여 추가
df_additional = df_cleaned.groupby('구_동').agg(
    횡단보도_개수=('구_동', 'size')  # 횡단보도 개수
).reset_index()

# 5. 상태, 횡단보도종류코드, 공사형태 데이터를 모두 병합
df_final = pd.merge(df_status, df_crosswalk_type, on='구_동', how='left')
df_final = pd.merge(df_final, df_construction_type, on='구_동', how='left')
df_final = pd.merge(df_final, df_additional, on='구_동', how='left')

# 열 이름 변경
df_final.columns = [
    '구_동', '상태_양호', '상태_파손', '상태_도색', '상태_노후',
    '종류_1', '종류_2', '종류_3', '종류_4',
    '공사_1', '공사_2', '공사_3', '공사_4', '공사_5', '공사_6', '공사_8', '공사_9', '공사_10',
    '횡단보도_개수'
]


# 상태, 횡단보도종류코드, 공사형태의 고유값 출력
unique_status = df_cleaned['상태'].unique()
unique_crosswalk_type = df_cleaned['횡단보도종류코드'].unique()
unique_construction_type = df_cleaned['공사형태'].unique()

print("상태 열의 고유값:", unique_status)
print("횡단보도종류코드 열의 고유값:", unique_crosswalk_type)
print("공사형태 열의 고유값:", unique_construction_type)

상태 열의 고유값: [1. 4. 3. 2.]
횡단보도종류코드 열의 고유값: [2. 3. 4. 1.]
공사형태 열의 고유값: ['005' '004' '003' '001' '010' '002' '006' '009' '008']


In [37]:
# 결과를 CSV 파일로 저장
df_final.to_csv('/content/drive/MyDrive/seoul_data/crosswalk_grouped_cleaned.csv', index=False)

# 결과 출력
print(df_final.head())
df_final.describe()

       구_동  상태_양호  상태_파손  상태_도색  상태_노후  종류_1  종류_2  종류_3  종류_4  공사_1  공사_2  \
0  강남구 개포동    379      0      0      1    69   244    60     7    11     0   
1  강남구 논현동    816      0      0      2   262   331   216     9   131     0   
2  강남구 대치동    852      0      0      5   202   421   227     7    43     0   
3  강남구 도곡동    381      0      0      4    70   227    79     9    68     0   
4  강남구 삼성동    779      0      0      2   171   439   166     5    57     1   

   공사_3  공사_4  공사_5  공사_6  공사_8  공사_9  공사_10  횡단보도_개수  
0    19   111   215     0     0     0     24      380  
1    12    81   574     0     0     0     20      818  
2    10   150   643     0     0     0     11      857  
3     8    68   236     0     0     0      5      385  
4    26   117   552     0     3     0     25      781  


,상태_양호,상태_파손,상태_도색,상태_노후,종류_1,종류_2,종류_3,종류_4,공사_1,공사_2,공사_3,공사_4,공사_5,공사_6,공사_8,공사_9,공사_10,횡단보도_개수
count,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000
mean,234.973684,0.013158,0.438596,1.826754,44.822368,132.442982,57.618421,2.368421,36.041667,0.037281,7.831140,47.848684,135.203947,0.129386,0.006579,0.506579,9.646930,237.252193
std,318.853494,0.131943,2.403423,3.762543,62.623111,186.813008,81.370724,4.264885,61.210575,0.249687,10.374525,67.410414,192.414468,0.665254,0.140488,3.898712,20.436993,321.588044
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,37.000000,0.000000,0.000000,0.000000,6.000000,18.000000,8.000000,0.000000,3.000000,0.000000,1.000000,5.000000,17.000000,0.000000,0.000000,0.000000,0.000000,37.000000
50%,91.000000,0.000000,0.000000,0.000000,17.000000,48.000000,21.000000,1.000000,12.000000,0.000000,3.000000,17.000000,49.000000,0.000000,0.000000,0.000000,1.000000,92.000000
75%,326.750000,0.000000,0.000000,2.000000,57.000000,173.750000,79.500000,3.000000,40.000000,0.000000,11.000000,67.250000,195.000000,0.000000,0.000000,0.000000,11.250000,327.750000
max,1931.000000,2.000000,32.000000,29.000000,387.000000,1292.000000,460.000000,37.000000,462.000000,3.000000,61.000000,463.000000,1364.000000,6.000000,3.000000,72.000000,214.000000,1946.000000


## 3. 노인보행자 사고 및 횡단보도 데이터 통합


In [38]:
# 1. 노인교통사고 엑셀 파일을 불러오고 필요하다면 CSV로 변환
traffic_accidents = pd.read_excel('/content/drive/MyDrive/seoul_data/노인교통사고.xlsx')
traffic_accidents.to_csv('/content/drive/MyDrive/seoul_data/elderly_traffic_accidents.csv', index=False)  # CSV로 변환

In [39]:
# 2. crosswalk_grouped_cleaned.csv 파일 불러오기
crosswalk_grouped = pd.read_csv('/content/drive/MyDrive/seoul_data/crosswalk_grouped_cleaned.csv')

# 3. 두 파일을 '구_동' 열을 기준으로 합치기 (outer join)
merged_data = pd.merge(traffic_accidents, crosswalk_grouped, on='구_동', how='outer')

In [40]:
# 4. 서로 다른 '구_동' 이름 찾기
traffic_accidents_gudong = set(traffic_accidents['구_동'].unique())
crosswalk_grouped_gudong = set(crosswalk_grouped['구_동'].unique())

# 노인교통사고 파일에만 있는 구_동
only_in_traffic = traffic_accidents_gudong - crosswalk_grouped_gudong
print("노인교통사고 파일에만 있는 구_동:", only_in_traffic)

# crosswalk_grouped_cleaned.csv 파일에만 있는 구_동
only_in_crosswalk = crosswalk_grouped_gudong - traffic_accidents_gudong
print("횡단보도 파일에만 있는 구_동:", only_in_crosswalk)

노인교통사고 파일에만 있는 구_동: {'종로구 익선동', '서대문구 냉천동', '서대문구 영천동', '서대문구 현저동', '서대문구 옥천동', '성북구 성북동1가', '강동구 강일동', '중구 명동1가'}
횡단보도 파일에만 있는 구_동: {'마포구 당인동', '중구 광희동1가', '용산구 용산동6가', '종로구 평동', '중구 남대문1가', '종로구 도렴동', '종로구 무악동', '중구 태평로1가', '중구 의주로2가', '종로구 공평동', '광진구 노유동', '중구 무교동', '종로구 명륜4가', '영등포구 문래동4가', '강동구 하일동', '종로구 원남동', '마포구 구수동', '강남구 포이동', '종로구 통인동', '서대문구 대신동', '종로구 종로1가', '중구 남산동3가', '마포구 하중동', '중구 남대문로2가', '중구 남산동1가', '영등포구 문래동5가', '종로구 체부동', '중구 정동', '중구 장교동', '용산구 용산동3가', '성북구 안암동3가', '종로구 효자동', '중구 산림동', '중구 수하동', '종로구 관훈동', '종로구 운니동', '종로구 원서동', '중구 의주로1가', '종로구 송현동', '중구 다동', '종로구 경운동', '강서구 오쇠동', '종로구 교북동', '중구 입정동', '마포구 마포동', '서초구 원지동', '종로구 신문로1가', '종로구 필운동', '용산구 서빙고동', '종로구 당주동', '강서구 오곡동', '중구 저동2가', '종로구 연지동', '종로구 동숭동', '성북구 안암동2가', '종로구 옥인동', '은평구 구파발동', '종로구 중학동', '중구 충무로5가', '중구 저동1가', '중구 서소문동', '중구 충정로1가', '중구 남학동', '종로구 송월동', '종로구 훈정동', '용산구 청파동2가', '종로구 누상동', '용산구 남영동', '종로구 신교동', '성북구 보문동3가', '종로구 적선동', '중구 주자동', '중구 충무로2가', '종로구 내수동', '광진구 모진동', '용산구 용산동4가', '용산

In [41]:
# 노인교통사고 파일에만 있는 구_동 목록
only_in_traffic = traffic_accidents_gudong - crosswalk_grouped_gudong

# 노인교통사고 파일에만 있는 구_동이 각각 몇 행을 가지고 있는지 계산
only_in_traffic_counts = traffic_accidents[traffic_accidents['구_동'].isin(only_in_traffic)].groupby('구_동').size()

# 결과 출력
print("노인교통사고 파일에만 있는 구_동의 행 개수:")
print(only_in_traffic_counts)


노인교통사고 파일에만 있는 구_동의 행 개수:
구_동
강동구 강일동      9
서대문구 냉천동     2
서대문구 영천동     1
서대문구 옥천동     2
서대문구 현저동     3
성북구 성북동1가    1
종로구 익선동      1
중구 명동1가      1
dtype: int64


In [42]:
# 각 구_동별로 몇 개의 행을 가지고 있는지 계산하고 내림차순으로 정렬
gudong_counts = traffic_accidents.groupby('구_동').size().sort_values(ascending=False)

# 결과 출력
print("각 구_동별 행 개수 (내림차순 정렬):")
print(gudong_counts)

# [인사이트]:노인교통사고 파일에만 있는 구_동 행의 개수는 크게 문제가 되지않을 것으로 보임. 지우겠음. 왜냐하면 어차피 지금 횡단보도 데이터에서 코드가 정확하지 않은 이슈가 있음.

각 구_동별 행 개수 (내림차순 정렬):
구_동
강서구 화곡동      136
관악구 신림동      121
중랑구 면목동      111
강북구 미아동      106
동대문구 제기동     100
            ... 
용산구 문배동        1
종로구 소격동        1
서대문구 영천동       1
종로구 팔판동        1
성북구 보문동4가      1
Length: 381, dtype: int64


In [43]:
# 노인교통사고 파일에만 있는 구_동에 해당하는 행 제거
traffic_accidents_cleaned = traffic_accidents[~traffic_accidents['구_동'].isin(only_in_traffic)]

# 결과 출력
print("노인교통사고 파일에만 있는 구_동 행 제거 후 데이터:")
print(traffic_accidents_cleaned.head())

노인교통사고 파일에만 있는 구_동 행 제거 후 데이터:
               사고번호       구_동    년도  월  일  사고시간       사고일시    년월  사망자수  중상자수  \
0  2021010200100072  성북구 하월곡동  2021  1  2    11 2021-01-02  2101     0     1   
1  2021010200100096   강북구 미아동  2021  1  2    12 2021-01-02  2101     0     0   
2  2021010200100173   관악구 신림동  2021  1  2    14 2021-01-02  2101     0     1   
3  2021010200100204   중랑구 중화동  2021  1  2    15 2021-01-02  2101     0     1   
4  2021010200100205   강서구 화곡동  2021  1  2    15 2021-01-02  2101     0     0   

   ...  사고유형  노면상태  기상상태  도로형태  도로형태상세  가해운전자 차종  피해운전자 성별  피해운전자 연령  \
0  ...     2     0     2     3       6        10         1        81   
1  ...     4     0     2     1       4         8         0        81   
2  ...     4     0     2     1       3         4         1        66   
3  ...     0     0     2     3       6        10         0        66   
4  ...     4     0     2     3       6         7         1        72   

   피해운전자 연령_cat  피해운전자 상해정도  
0           3.0          

In [45]:
# 두 파일을 '구_동' 열을 기준으로 outer join으로 다시 병합
merged_data = pd.merge(traffic_accidents_cleaned, crosswalk_grouped, on='구_동', how='outer')

# 결과 출력
print("노인교통사고 파일과 횡단보도 파일을 병합한 데이터:")
print(merged_data.head())

노인교통사고 파일과 횡단보도 파일을 병합한 데이터:
           사고번호       구_동      년도    월     일  사고시간       사고일시      년월  사망자수  \
0  2.021010e+15  성북구 하월곡동  2021.0  1.0   2.0  11.0 2021-01-02  2101.0   0.0   
1  2.021011e+15  성북구 하월곡동  2021.0  1.0   5.0  10.0 2021-01-05  2101.0   0.0   
2  2.021012e+15  성북구 하월곡동  2021.0  1.0  19.0  14.0 2021-01-19  2101.0   0.0   
3  2.021041e+15  성북구 하월곡동  2021.0  4.0  13.0   6.0 2021-04-13  2104.0   0.0   
4  2.021042e+15  성북구 하월곡동  2021.0  4.0  19.0  18.0 2021-04-19  2104.0   0.0   

   중상자수  ...  공사_1  공사_2  공사_3  공사_4  공사_5  공사_6  공사_8  공사_9  공사_10  횡단보도_개수  
0   1.0  ...   105     0     5    91   360     0     0     0     48      609  
1   1.0  ...   105     0     5    91   360     0     0     0     48      609  
2   0.0  ...   105     0     5    91   360     0     0     0     48      609  
3   1.0  ...   105     0     5    91   360     0     0     0     48      609  
4   0.0  ...   105     0     5    91   360     0     0     0     48      609  

[5 rows x 41 columns]

In [47]:
print(merged_data.columns)
merged_data.describe()

Index(['사고번호', '구_동', '년도', '월', '일', '사고시간', '사고일시', '년월', '사망자수', '중상자수',
       '경상자수', '부상신고자수', '사상자수', '사고유형', '노면상태', '기상상태', '도로형태', '도로형태상세',
       '가해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 연령_cat', '피해운전자 상해정도',
       '상태_양호', '상태_파손', '상태_도색', '상태_노후', '종류_1', '종류_2', '종류_3', '종류_4',
       '공사_1', '공사_2', '공사_3', '공사_4', '공사_5', '공사_6', '공사_8', '공사_9', '공사_10',
       '횡단보도_개수'],
      dtype='object')


,사고번호,년도,월,일,사고시간,사고일시,년월,사망자수,중상자수,경상자수,...,공사_1,공사_2,공사_3,공사_4,공사_5,공사_6,공사_8,공사_9,공사_10,횡단보도_개수
count,5.449000e+03,5449.000000,5449.000000,5449.000000,5449.000000,5449,5449.000000,5449.000000,5449.000000,5449.000000,...,5532.000000,5532.000000,5532.000000,5532.000000,5532.000000,5532.00000,5532.000000,5532.000000,5532.000000,5532.000000
mean,2.022115e+15,2022.046247,6.688383,15.641769,13.326849,2022-07-23 17:43:09.172325120,2211.313085,0.032300,0.512938,0.451643,...,103.701193,0.100687,19.263377,138.057484,398.423536,0.35846,0.005965,1.897144,28.825560,690.633406
min,2.021010e+15,2021.000000,1.000000,1.000000,0.000000,2021-01-02 00:00:00,2101.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000
25%,2.021102e+15,2021.000000,4.000000,8.000000,10.000000,2021-10-23 00:00:00,2110.000000,0.000000,0.000000,0.000000,...,30.000000,0.000000,8.000000,59.000000,198.000000,0.00000,0.000000,0.000000,6.000000,351.000000
50%,2.022073e+15,2022.000000,7.000000,16.000000,13.000000,2022-07-31 00:00:00,2207.000000,0.000000,1.000000,0.000000,...,82.000000,0.000000,17.000000,121.000000,329.000000,0.00000,0.000000,0.000000,17.000000,584.000000
75%,2.023043e+15,2023.000000,10.000000,23.000000,17.000000,2023-04-29 00:00:00,2304.000000,0.000000,1.000000,1.000000,...,139.000000,0.000000,29.000000,182.000000,587.000000,0.00000,0.000000,0.000000,36.000000,959.000000
max,2.023123e+15,2023.000000,12.000000,31.000000,23.000000,2023-12-31 00:00:00,2312.000000,2.000000,6.000000,9.000000,...,462.000000,3.000000,61.000000,463.000000,1364.000000,6.00000,3.000000,72.000000,214.000000,1946.000000
std,8.162304e+11,0.814510,3.433576,8.752205,4.770636,NaN,81.622443,0.178875,0.524604,0.563759,...,95.603462,0.447359,13.938116,103.435347,289.203199,1.10571,0.133654,7.850949,33.992679,475.288904


In [51]:
# 각 구_동별 교통사고 건수 계산
merged_data['교통사고_건수'] = merged_data.groupby('구_동')['구_동'].transform('size')

# 결과 출력
print(merged_data[['구_동', '교통사고_건수']].drop_duplicates().head())


          구_동  교통사고_건수
0    성북구 하월곡동       36
36    강북구 미아동      106
142   관악구 신림동      121
263   중랑구 중화동       43
306   강서구 화곡동      136


### 2-1. 교통사고가 없는 지역 행을 따로 추출하고 제거

In [52]:
# 교통사고 관련 열을 확인하여 모든 값이 NA인 경우 필터링
accident_columns = ['사고번호', '년도', '월', '일', '사고시간', '사고일시', '년월',
                    '사망자수', '중상자수', '경상자수', '부상신고자수', '사상자수',
                    '사고유형', '노면상태', '기상상태', '도로형태', '도로형태상세',
                    '가해운전자 차종', '피해운전자 성별', '피해운전자 연령',
                    '피해운전자 연령_cat', '피해운전자 상해정도']

non_accident_crossroad_columns = ['상태_양호', '상태_파손', '상태_도색', '상태_노후',
                        '종류_1', '종류_2', '종류_3', '종류_4',
                        '공사_1', '공사_2', '공사_3', '공사_4', '공사_5', '공사_6',
                        '공사_8', '공사_9', '공사_10', '횡단보도_개수']

# 교통사고 관련 열은 NA지만, 상태/종류/공사 관련 열에 값이 있는 행 필터링
non_accident_rows = merged_data[merged_data[accident_columns].isna().all(axis=1) &
                                merged_data[non_accident_crossroad_columns].notna().any(axis=1)]

# 그 행들의 개수 출력
print(f"교통사고가 없는 지역 행 개수: {len(non_accident_rows)}")

# 교통사고가 없는 지역을 CSV로 저장
non_accident_rows.to_csv('/content/drive/MyDrive/seoul_data/non_accident_areas.csv', index=False)

# 교통사고가 없는 지역을 merged_data에서 제거
merged_data = merged_data.drop(non_accident_rows.index)

# 행 개수 출력
print(f"교통사고가 없는 지역을 제거한 후의 행 개수: {len(merged_data)}")


교통사고가 없는 지역 행 개수: 83
교통사고가 없는 지역을 제거한 후의 행 개수: 5449


In [54]:
# 각각의 열에 NA 값이 있는지 확인
na_counts = merged_data.isna().sum()
# 피해운전자 연령_cat이 NaN인 행을 삭제
merged_data_cleaned = merged_data.dropna(subset=['피해운전자 연령_cat'])

# 삭제 후 데이터 개수 출력
print(f"피해운전자 연령_cat 결측치가 있는 행을 삭제한 후의 행 개수: {len(merged_data_cleaned)}")

피해운전자 연령_cat 결측치가 있는 행을 삭제한 후의 행 개수: 5410


🤔 5410개의 데이테가 군집화 및 예측 모델링 수행하는 데 충분한 크기의 데이터셋일까?
<br>
➡️ 충분히 사용할 수 있는 크기이다. K-Means는 상대적으로 작은 데이터셋에서 잘 작동한다.
<br>
➡️ 대신 특성수가 너무 많으면 군집화 성능이 떨어지니까 차원축소를 잘해야 한다.

## 3. Min-Max 정규화
- 변수들의 범위가 매우 다를 때, 특히 모든 값이 양수일 때 min-max 정규화를 사용하면 각 변수를 동일한 범위로 맞출 수 있어서 모델 학습에 더 적합해진다.
`예) 사상자수는 1 ~ 11사이의 값인데, 횡단보도개수는 1 ~ 2139개와 같이 큰 차이가 있기에 각 변수를 0~1 사이로 변환`

In [55]:
# 어떤 변수가 있는지 보자!
merged_data_cleaned.columns

Index(['사고번호', '구_동', '년도', '월', '일', '사고시간', '사고일시', '년월', '사망자수', '중상자수',
       '경상자수', '부상신고자수', '사상자수', '사고유형', '노면상태', '기상상태', '도로형태', '도로형태상세',
       '가해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 연령_cat', '피해운전자 상해정도',
       '상태_양호', '상태_파손', '상태_도색', '상태_노후', '종류_1', '종류_2', '종류_3', '종류_4',
       '공사_1', '공사_2', '공사_3', '공사_4', '공사_5', '공사_6', '공사_8', '공사_9', '공사_10',
       '횡단보도_개수', '교통사고_건수'],
      dtype='object')

In [64]:
# '피해운전자 연령'이 '12이하'인 행을 제거
merged_data_cleaned = merged_data_cleaned[merged_data_cleaned['피해운전자 연령'] != '12 이하']

In [65]:
# 결과 확인
print(f"'12이하' 연령의 행을 제거한 후의 데이터 크기: {len(merged_data_cleaned)}")

'12이하' 연령의 행을 제거한 후의 데이터 크기: 5408


In [66]:
# 범주형 변수 목록
categorical_columns = [
    '사고유형', '노면상태', '기상상태', '도로형태', '도로형태상세',
    '가해운전자 차종', '피해운전자 성별', '피해운전자 연령_cat', '구_동'
]

# 각 범주형 변수에 대해 범주별 개수 출력
for col in categorical_columns:
    value_counts = merged_data_cleaned[col].value_counts()
    print(f"\n{col}의 각 범주별 개수:")
    print(merged_data_cleaned[col].value_counts())
    # 각 범주 개수의 총합 출력
    total_count = value_counts.sum()
    print(f"{col}의 범주별 개수의 총합: {total_count}")


사고유형의 각 범주별 개수:
사고유형
0.0    2312
4.0    1757
3.0     646
2.0     417
1.0     276
Name: count, dtype: int64
사고유형의 범주별 개수의 총합: 5408

노면상태의 각 범주별 개수:
노면상태
0.0    4884
4.0     426
1.0      72
2.0      17
3.0       6
5.0       2
6.0       1
Name: count, dtype: int64
노면상태의 범주별 개수의 총합: 5408

기상상태의 각 범주별 개수:
기상상태
2.0    4787
3.0     322
4.0     199
0.0      83
1.0      17
Name: count, dtype: int64
기상상태의 범주별 개수의 총합: 5408

도로형태의 각 범주별 개수:
도로형태
3.0    2721
1.0    2125
2.0     523
4.0      28
0.0      11
Name: count, dtype: int64
도로형태의 범주별 개수의 총합: 5408

도로형태상세의 각 범주별 개수:
도로형태상세
6.0    3208
4.0     892
5.0     629
3.0     604
7.0      28
8.0      13
0.0      11
2.0      11
9.0       9
1.0       3
Name: count, dtype: int64
도로형태상세의 범주별 개수의 총합: 5408

가해운전자 차종의 각 범주별 개수:
가해운전자 차종
4.0     2935
10.0     811
5.0      560
7.0      437
8.0      341
2.0      111
0.0       87
6.0       61
1.0       54
9.0       10
3.0        1
Name: count, dtype: int64
가해운전자 차종의 범주별 개수의 총합: 5408

피해운전자 성별의 각 범주별 개수:
피해운전자 성별

In [67]:
# 정규화가 필요한 숫자형 변수 목록
numeric_columns = [
    '사망자수', '중상자수', '경상자수', '부상신고자수', '사상자수', '피해운전자 연령',
    '상태_양호', '상태_파손', '상태_도색', '상태_노후', '종류_1', '종류_2', '종류_3',
    '종류_4', '공사_1', '공사_2', '공사_3', '공사_4', '공사_5', '공사_6', '공사_8',
    '공사_9', '공사_10', '횡단보도_개수', '교통사고_건수'
]

# Min-Max Scaler 생성
scaler = MinMaxScaler()

# 기존 데이터를 유지하고, 정규화된 데이터를 새로운 scaled 열에 저장
for col in numeric_columns:
    merged_data_cleaned[f'{col}_scaled'] = scaler.fit_transform(merged_data_cleaned[[col]])

# 결과 확인
print("정규화된 데이터가 새로운 '_scaled' 열로 추가되었습니다.")
print(merged_data_cleaned[[f'{col}_scaled' for col in numeric_columns]].head())


정규화된 데이터가 새로운 '_scaled' 열로 추가되었습니다.
   사망자수_scaled  중상자수_scaled  경상자수_scaled  부상신고자수_scaled  사상자수_scaled  \
0          0.0          0.2          0.0            0.0          0.0   
1          0.0          0.2          0.0            0.0          0.0   
2          0.0          0.0          0.2            0.0          0.0   
3          0.0          0.2          0.0            0.0          0.0   
4          0.0          0.0          0.2            0.0          0.0   

   피해운전자 연령_scaled  상태_양호_scaled  상태_파손_scaled  상태_도색_scaled  상태_노후_scaled  \
0           1.0000      0.312435           0.0           0.0      0.172414   
1           0.9375      0.312435           0.0           0.0      0.172414   
2           0.4375      0.312435           0.0           0.0      0.172414   
3           0.2500      0.312435           0.0           0.0      0.172414   
4           0.5000      0.312435           0.0           0.0      0.172414   

   ...  공사_2_scaled  공사_3_scaled  공사_4_scaled  공사_5_scaled  공사

In [68]:
# scaled가 완료된 데이터셋을 CSV 파일로 저장
merged_data_cleaned.to_csv('/content/drive/MyDrive/seoul_data/merged_data_cleaned_scaled.csv', index=False)

print("데이터셋이 성공적으로 저장되었습니다.")

데이터셋이 성공적으로 저장되었습니다.


➡️ 자!! 여기까지 전처리 및 스케일링이 마무리된 상태입니다! na 값 다 처리하였고 스케일도 맞췄습니다! 이제는 데이터셋 특징을 파악해볼거에요!! 🤔

## 4. 탐색적 데이터 분석
- 기초 통계 분석
- 범주형 변수에 대한 상관관계 분석 (카이제곱 검정)
- 숫자형 변수에 대한 상관관계 분석

### 4-1. 통계요약

In [ ]:
# 수치형 데이터의 통계 요약
print("수치형 변수에 대한 통계 요약:")
print(merged_data_cleaned.describe())

# 범주형 데이터의 통계 요약
print("범주형 변수에 대한 통계 요약:")
print(merged_data_cleaned.describe(include=['object', 'category']))
## 지금 이거 약간 변경 필요함.

### 4-2. 결측치 확인

In [70]:
# 각 열의 결측치 개수 확인
na_counts = merged_data_cleaned.isna().sum()

print("각 열의 결측치 개수:")
print(na_counts)

각 열의 결측치 개수:
사고번호              0
구_동               0
년도                0
월                 0
일                 0
                 ..
공사_8_scaled       0
공사_9_scaled       0
공사_10_scaled      0
횡단보도_개수_scaled    0
교통사고_건수_scaled    0
Length: 66, dtype: int64
